In [8]:
import os
import sys
import requests
from tqdm import tqdm

model = '345M'

subdir = os.path.join('models', model)
if not os.path.exists(subdir):
    os.makedirs(subdir)
subdir = subdir.replace('\\','/') # needed for Windows

for filename in ['checkpoint','encoder.json','hparams.json','model.ckpt.data-00000-of-00001', 'model.ckpt.index', 'model.ckpt.meta', 'vocab.bpe']:

    r = requests.get("https://storage.googleapis.com/gpt-2/" + subdir + "/" + filename, stream=True)

    with open(os.path.join(subdir, filename), 'wb') as f:
        file_size = int(r.headers["content-length"])
        chunk_size = 1000
        with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
            # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)
                pbar.update(chunk_size)

Fetching checkpoint: 1.00kit [00:00, 315kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 25.6Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 429kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:57, 24.5Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 2.87Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 28.0Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 24.9Mit/s]                                                       


In [9]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-general')

In [12]:
from glob import glob
from tqdm import tqdm

files = glob('models/345M/*')
for file in tqdm(files):
    f = file.split('models/345M/')[1]
    blob = bucket.blob(f'gpt2-345M-pretrained/{f}')
    blob.upload_from_filename(file)

100%|██████████| 7/7 [00:13<00:00,  1.90s/it]
